# Pneumonia-Treatment-RAG

In [1]:
!pip install PyPDF2
# !pip install pdfreader
!pip install transformers torch
# !pip install gensim
# !pip install --upgrade lamini
!pip install sentence_transformers
# !pip install python-docx
# !pip install pickle5
# !pip install faiss-cpu rank_bm25 numpy
!pip install sentencepiece
# !pip install subword-nmt
# !pip install tiktoken
# !pip install bpemb
!pip install langchain_openai 
# !pip install langchain 
!pip install openai 
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 8.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
  

# **SentenceTransformer Embedding **

In [2]:
from sentence_transformers import SentenceTransformer
def get_sentenceTF_embeddings(sentences):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings =[]
  for chunk in sentences:
    embeddings.append(model.encode(chunk))
  print(len(embeddings))
  return embeddings

def Embed_stenteceTF(sentence):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  return model.encode(sentence)

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-18 10:31:57.884538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 10:31:57.884695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 10:31:58.031115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Sementic Chuncking**

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans


def get_bert_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize sentences
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token representation as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    return embeddings

def semantic_chunking_with_attention(text, n_clusters=5, max_chunk_size=512):
    # Split the input text into sentences based on '.'
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

    # Get BERT embeddings for sentences
    embeddings = get_bert_embeddings(sentences)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(embeddings)
    labels = kmeans.labels_

    # Group sentences by cluster
    clusters = {}
    for sentence, label in zip(sentences, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sentence)

    # Create chunks from clusters with max_chunk_size constraint
    chunks = []
    for cluster in clusters.values():
        current_chunk = ""
        for sentence in cluster:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for the period
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if current_chunk:
            chunks.append(current_chunk.strip())

    return chunks

# **Recursive Chunking**

In [4]:
def division_Chunk(text, chunk_size=400):
    sentences = text.split(".")
    chunk1 = ""
    chunks = []

    for sentence in sentences:
        word_count = len(sentence.split())

        if word_count > chunk_size:
            chunks.extend(Recursive_Chunking(sentence, chunk_size))
        else:
            if len(chunk1.split()) + word_count <= chunk_size:
                chunk1 += sentence + "\n"
            else:
                chunks.append(chunk1.strip())
                chunk1 = sentence + "\n"

    if chunk1.strip():
        chunks.append(chunk1.strip())

    return chunks

def Recursive_Chunking(text, chunk_size=1000):
    chunks = text.split("\n\n")
    #print(len(chunks))
    result_chunks = []

    for chunk in chunks:
        word_count = len(chunk.split())

        if word_count > chunk_size:
            middle = int(0.8 * word_count)
            words = chunk.split()
            part1 = " ".join(words[:middle])
            part2 = " ".join(words[middle:])

            result_chunks.extend(division_Chunk(part1, chunk_size))
            result_chunks.extend(division_Chunk(part2, chunk_size))
        else:
            result_chunks.append(chunk.strip())

    return result_chunks

# Add_Data_From_PDF_file

In [5]:
from PyPDF2 import PdfReader
import re

def read_pdf_file(file_path):
    points = []
    # Open the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                points.append(text.strip())
    return points

def replace_double_spaces(text):
    # Replace all occurrences of non-breaking spaces and zero-width spaces
    cleaned_text = re.sub(r'\xa0', ' ', text)
    cleaned_text1 = re.sub(r'\u200b', '', cleaned_text)
    return cleaned_text1

def process_pdf_files(file_paths, chunking_type, max_chunk_size, number_of_clusters=20):
    all_embeddings = []
    all_chunks = []

    for file_path in file_paths:
        print(f"Processing: {file_path}")
        Pneumonia = read_pdf_file(file_path)
        
        # Clean the text
        for i in range(len(Pneumonia)):
            Pneumonia[i] = replace_double_spaces(Pneumonia[i])
        
        Pneumonia_Str = "\n".join(Pneumonia)
        
        # Chunking
        if chunking_type == "Recursive":
            chunks = Recursive_Chunking(Pneumonia_Str, chunk_size=max_chunk_size)
        elif chunking_type == "Semantic":
            chunks = semantic_chunking_with_attention(Pneumonia_Str, n_clusters=number_of_clusters, max_chunk_size=max_chunk_size)
        
        # Get embeddings
        embedding = get_sentenceTF_embeddings(chunks)
        
        all_embeddings.extend(embedding)
        all_chunks.extend(chunks)
    
    return all_embeddings, all_chunks

# **PrepareData**

In [6]:
# Provide the list of PDF file paths
file_paths = [
    '/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf',
    '/kaggle/input/pneumoniadocuments/Advanced_Physiology_and_Pathophysiology_Essentials..._----_(Chapter_5_Infectious_Disease) (1).pdf',
    '/kaggle/input/pneumoniadocuments/pneumonia-unmasked-unlocking-the-secrets.pdf'
]

# Process all the files and obtain embeddings and chunks
embeddings, chunks = process_pdf_files(file_paths, chunking_type='Semantic', max_chunk_size=600, number_of_clusters=20)

# Print the lengths of the chunks and embeddings
print(len(chunks))
print(len(embeddings))

Processing: /kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

26
1036
1036


In [7]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(chunks, embeddings))
]

In [11]:
from qdrant_client import QdrantClient
import json

with open('config.json', 'r') as file:
    config = json.load(file)


url = config['qdrant_url']
api_key = config['qdrant_api_key']

# Initialize Qdrant client
qdrant_client = QdrantClient(url=url, api_key=api_key)

In [12]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 384  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="pneumonoBot",
    vectors_config=vector_params  # Correct argument name
)

True

In [13]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="pneumonoBot",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# llm_model.generate(concatenated_text,max_tokens=2048,max_new_tokens=2048 )

In [14]:
query = ["How to treat pneumonia"]
query_embedding_response = get_sentenceTF_embeddings(query)
# print(query_embedding_response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1


In [15]:
# Assuming query_embedding_response is a list of lists or a NumPy array
query_embedding = query_embedding_response[0].tolist()  # Convert to list if necessary

In [16]:
# Perform similarity search
results = qdrant_client.search(
    collection_name="pneumonoBot",
    query_vector=query_embedding,
)
# Print results
for result in results:
    print(result)

id='3aa9b064-1294-43fe-9a5a-cea907bbab43' version=0 score=0.68566144 payload={'text': 'Despite \navailability of effective antibiotic treatment the mortality remains 6 to 12% in patient s \nadmitted to hospital with community acquired pneumonia and over 50% in patients admitted \nto the intensive care unit (figures from the United Kingdom). Supportive treatment explored in patients with pneumonia includes oxygen in a \nvariety of applications, fluid therapy regimens, chest physiotherapy, steroid treatment, \ngranulocyte colony stimulating factor treatment, surfactant application and vitamins A, C and \nD, zinc, protein and calorie supplements.'} vector=None shard_key=None order_value=None
id='7fdda19d-4e61-455d-8221-7a054e8bca8c' version=0 score=0.6829466 payload={'text': 'The \nobjective of this review is to summarize evidence  for the effectiveness of modes of \nsupportive treatment in patients with pneumonia. It is the largest killer, accounting  \nfor 28 to 34% of all child deaths 

In [17]:
# NVIDIA NIM API
from openai import OpenAI
import json

with open('config.json', 'r') as file:
    config = json.load(file)

client = OpenAI(
  base_url = config['nvidia_base_url'],
  api_key = config['nvidia_api_key']
)

In [18]:
def custom_prompt(query: str, results):
    # Extract the page content from the results
    source_knowledge = "\n".join([x.payload['text'] for x in results])
    
    # Create the augmented prompt
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""
    
    return augment_prompt

In [63]:
# print(custom_prompt(query, results))

In [19]:
messages = []
# messages.append({"role":"assistant", "content": full_response})
prompt = {"role":"system", "content": custom_prompt(query, results)}
messages.append(prompt)

res = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=messages,
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

full_response = ""
for chunk in res:
  if chunk.choices[0].delta.content is not None:
#     print(chunk.choices[0].delta.content, end="")
    full_response += chunk.choices[0].delta.content
print(full_response)

According to the provided contexts, the treatment of pneumonia involves both antibiotic therapy and supportive treatment. Here are some of the ways pneumonia can be treated:

1. Antibiotic therapy: This is the primary treatment for bacterial pneumonia. Antibiotics can help kill the bacteria causing the infection.
2. Supportive treatment: This includes various measures to help the patient recover from pneumonia. Some of the supportive treatments mentioned include:
	* Oxygen therapy: to help increase oxygen levels in the blood
	* Fluid therapy: to prevent dehydration and maintain fluid balance
	* Chest physiotherapy: to help loosen and clear mucus from the lungs
	* Steroid treatment: to reduce inflammation in the lungs
	* Granulocyte colony-stimulating factor treatment: to stimulate the production of white blood cells
	* Surfactant application: to help reduce the surface tension in the lungs and make it easier to breathe
	* Vitamins A, C, and D, zinc, protein, and calorie supplements: to